In [1]:
!pip install --upgrade pyiwn
!pip install --upgrade xlrd


     |████████████████████████████████| 96 kB 4.1 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


Importing pyiwn API to access Indowordnet and Loading Kannada language


In [2]:
import pyiwn
iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.KANNADA)

2022-04-02:03:59:24,131 INFO     [helpers.py:20] Downloading IndoWordNet data of size ~31 MB...


[██████████████████████████████████████████████████]

2022-04-02:03:59:25,707 INFO     [helpers.py:43] Extracting /root/iwn_data.tar.gz into /root...


2022-04-02:03:59:26,906 INFO     [helpers.py:48] Removing temporary zip file from /root/iwn_data.tar.gz
2022-04-02:03:59:26,913 INFO     [helpers.py:51] IndoWordNet data successfully downloaded at /root/iwn_data
2022-04-02:03:59:26,993 INFO     [utils.py:159] NumExpr defaulting to 2 threads.
2022-04-02:03:59:27,318 INFO     [iwn.py:43] Loading kannada language synsets...


Lesk Algorithm- Core
(For Visual)

In [3]:
def Lesk(word, sentence,postag=""):
    senses = get_senses(word)
    if postag != "":
      #senses = get_senses(word)
      senses=specified_sense(word,postag)
    best_sense_word = senses[0]
    max_overlap = 0
    context = get_words(sentence)
    
    for sense in senses:
        signature = get_gloss_and_examples(sense)
        overlap = compute_overlap(signature, context)
        
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense_word = sense
    
    return (best_sense_word,best_sense_word.lemma_names(),best_sense_word.examples())

Lesk Algorithm- Core
(For WSD System)

In [4]:
def Lesk1(word, sentence,postag=""):
    senses = get_senses(word)
    if postag != "":
      #senses = get_senses(word)
      senses=specified_sense(word,postag)
    best_sense_word = senses[0]
    max_overlap = 0
    context = get_words(sentence)
    
    for sense in senses:
        signature = get_gloss_and_examples_synonyms(sense)
        overlap = compute_overlap(signature, context)
        
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense_word = sense
    
    return best_sense_word.synset_id()

Function to get all senses of a word

In [5]:
def get_senses(word):
    return iwn.synsets(word)

Function to get specified senses using POSTag method

In [6]:
def specified_sense(word, postag):
  all_senses = get_senses(word)
  specific_senses = []
  for sense in all_senses:
    pos = sense.pos().upper()
    postag = postag.upper()

    if postag == pos:
      specific_senses.append(sense)
    
  if len(specific_senses) == 0:
    return all_senses
  return specific_senses

Function to Split a Sentence into words

In [7]:
def get_words(sentence):
  return sentence.split(" ")

Function to get Gloss and Examples of a word from the Indowordnet

In [8]:
def get_gloss_and_examples(sense):
  gloss = get_words(sense.gloss())
  examples = sense.examples()
  meaning = []
  for example in examples:
    meaning.extend(get_words(example))

  gloss_and_examples = []
  gloss_and_examples.extend(gloss)
  gloss_and_examples.extend(meaning)
  return gloss_and_examples

Function to get Gloss, Examples and synonyms of a word from the Indowordnet

In [9]:
def get_gloss_and_examples_synonyms(sense):
  gloss = get_words(sense.gloss())
  synonyms = sense.lemma_names()
  examples = sense.examples()
  meaning = []
  for example in examples:
    meaning.extend(get_words(example))
  gloss_and_examples1 = []
  gloss_and_examples1.extend(gloss)
  gloss_and_examples1.extend(meaning)
  gloss_and_examples1.extend(synonyms)
  return gloss_and_examples1

Function to compute similarity between given sentence and Gloss_examples

In [10]:
def compute_overlap(signature, context):
  overlap = 0
  for word in context:
    if word in signature:
      overlap += 1
  
  return overlap

Implementing Lesk

In [11]:
Lesk("ಅಲೆ","ಗಂಟೆಯ ನಿನಾದ ಅಲೆ ಅಲೆಯಾಗಿ ಕೇಳಿ ಬಂತು",'noun')

(Synset('ಅಲೆ.noun.3332'),
 ['ಅಲೆ', 'ಲಹರಿ', 'ತರಂಗ'],
 ['ನೀರಿನ ಅಲೆಗಳನ್ನು ನೋಡಿ ಕವಿಯು ಕವಿತೆಯನ್ನು ಬರೆದನು.'])

In [12]:
Lesk("ಕರ","ನಾನು ಸರಕಾರಕ್ಕೆ ಕರವನ್ನು ಕೊಟ್ಟೆ")

(Synset('ತೆರಿಗೆ_ಸುಂಕ.noun.6353'),
 ['ತೆರಿಗೆ_ಸುಂಕ', 'ಕರ', 'ಕಂದಾಯ'],
 ['ಪ್ರತಿವರ್ಷ ನಾನು ಇಪ್ಪತ್ತು ಸಾವಿರದಷ್ಟು ತೆರಿಗೆಯನ್ನು ಭರಿಸಬೇಕಾಗಿ ಬರುವುದು.'])

In [13]:
Lesk("ಕರ","ಮುಖದ ತುಂಬಾ ಧೂಳು ತಾಗಿದ್ದರಿಂದ ನಾನು ಕರ ವಸ್ತ್ರದಿಂದ ಮುಖ ಒರೆಸಿಕೊಂಡೆ")

(Synset('ಭುಜ.noun.491'),
 ['ಭುಜ', 'ಕೈ', 'ಕರ', 'ತೋಳು'],
 ['ಭೀಮನ ಭುಜದಲ್ಲಿ ತುಂಬಾ ಬಲವಿತ್ತು'])

In [14]:
Lesk("ಕರಿ","ನಾನು ಎಣ್ಣೆಯಲ್ಲಿ ಕರಿದ ತಿಂಡಿಗಳನ್ನು ಹೆಚ್ಚಾಗಿ ತಿನ್ನುತ್ತೇನೆ")

(Synset('ಕರಿ.verb.5343'),
 ['ಕರಿ'],
 ['ಅವಳು ಸಂಜೆ ಬರುವ ಅತಿಥಿಗಳಿಗೆಂದು ವಡೆ ಮತ್ತು ಪಕೋಡ ಕರಿಯುತ್ತಿದ್ದಾಳೆ.'])

In [15]:
Lesk("ಮುತ್ತು","ನನ್ನ ಹತ್ತಿರ ಮುತ್ತಿನ ಸರ ಇದೆ")

(Synset('ಮುತ್ತು.noun.1201'), ['ಮುತ್ತು'], ['ಗೀತ ಮುತ್ತಿನ ಉಂಗುರವನ್ನು ಧರಿಸಿದ್ದಳು'])

In [16]:
Lesk("ಅಣಿ","ಒಂದು ಡಯಲ್-ಅಪ್-ಜಾಲಬಂಧ ಸಂಪರ್ಕವನ್ನು ಅಣಿಗೊಳಿಸುವ ಮೊದಲು ಪೂರ್ವನಿಯೋಜಿತವಾದ ಬ್ಲೂಟೂತ್ ಅಡಾಪ್ಟರ್ ಅನ್ನು ಶಕ್ತಗೊಳಿಸಬೇಕು")

(Synset('ತಯಾರಿ.noun.1416'),
 ['ತಯಾರಿ', 'ಸಿದ್ದತೆ', 'ಸಜ್ಜು', 'ಅಣಿ', 'ಸನ್ನಾಹ', 'ಸವರಣೆ'],
 ['ಸೀಮಾಳ ಮದುವೆಗಾಗಿ ಬಹಳ ಜೋರಾಗಿ ತಯಾರಿ ನಡೆಯುತ್ತಿದೆ.'])

Reading  Test data from the Google Drive

In [17]:
!pip install --upgrade -q gspread



In [21]:
import gspread
import pandas as pd
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()




2022-04-02:04:02:31,191 WARNING  [_default.py:484] No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


In [19]:
auth.authenticate_user()


2022-04-02:03:59:32,287 INFO     [auth.py:98] Failure refreshing credentials: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Enginemetadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7f38e7f8f250>)
2022-04-02:03:59:32,391 INFO     [auth.py:98] Failure refreshing credentials: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Enginemetadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7f38e7f65090>)


In [23]:
#gc = gspread.authorize(GoogleCredentials.get_application_default())

gc = gspread.authorize(creds)

In [24]:
worksheet = gc.open('WSD_Test_Dataset_copi').sheet1

In [25]:
rows = worksheet.get_all_values()
dataset = pd.DataFrame.from_records(rows)

Implementing WSD System on the Test Data

In [26]:
def wsd(rows):
  sys_id_list = []
  for i in range(1,len(rows)):
    tw = rows[i][0]
    sen = rows[i][1]
    sys_id = Lesk1(tw,sen)
    rows[i][5] = str(sys_id)
  return rows

Running WSD System

In [27]:
wsd(rows)

[['Target Word',
  'Sentence/Gloss',
  'Correct ID1',
  'Correct ID2',
  'Correct ID3',
  'Calculated ID'],
 ['ಮಧು',
  'ಪ್ರಶಸ್ತಿ ಸಮಾರಂಭದಲ್ಲಿ ಶಾರುಖ್ ಖಾನ್ ಅವರ ಮಧುಚಂದ್ರದ ಹೆಂಡತಿ ಗೌರಿ ಖಾನ್ನೊಂದಿಗೆ ಉಲ್ಲಾಸದ ಥ್ರೋಬ್ಯಾಕ್ ಕಥೆಯನ್ನು ಬಹಿರಂಗಪಡಿಸಿದರು',
  '8047',
  '',
  '',
  '3170'],
 ['ಮಧು',
  'ಮನಸಿನಲ್ಲಿ ಯವುದೋ ಬೇಸರ ಇದ್ದ ಕಾರಣ ಆತ ಇಡೀ ದಿನವನ್ನು ಮಧು ಹೀರುತ್ತಾ ಕಳೆದನು',
  '7522',
  '',
  '',
  '3170'],
 ['ಮಧು',
  'ಮಧುಮೇಹ, ರಕ್ತದೊತ್ತಡ, ಕೊಲೆಸ್ಟ್ರಾಲ್, ಎಸ್ ಜಿ ಪಿ ಟಿ (ಲಿವರ್ ಸ್ಕ್ರೀನಿಂಗ್), ಕ್ರಿಯೇಟಿನೈನ್ (ಮೂತ್ರಪಿಂಡಗಳ ಪರೀಕ್ಷೆ) ಮುಂತಾದ ಪರೀಕ್ಷೆಗಳು ಶಿಬಿರಾರ್ಥಿಗಳಿಗೆ ಉಚಿತವಾಗಿ ಲಭ್ಯವಿದ್ದವು',
  '8047',
  '',
  '',
  '3170'],
 ['ಮಧು', 'ದೇವತೆಗಳು ಮಧುಪಾನ ಮಾಡಿ ಅಮರರಾದರು', '3170', '', '', '3170'],
 ['ಯಂತ್ರ',
  'ಸ್ಟೇಟ್ ಬ್ಯಾಂಕ್ ಆಫ್ ಇಂಡಿಯಾವು ನಗದು ಹಣದಿಂದ ಪಿಒಎಸ್ ಯಂತ್ರದ ಮೂಲಕ ಈ ಸೌಲಭ್ಯವನ್ನು ಒದಗಿಸಿ ಬಿಗ್ ಬಜಾರ್ಗೆ ಸಹಾಯ ಮಾಡಿದೆ',
  '4168',
  '',
  '',
  '4168'],
 ['ಯಂತ್ರ',
  'ಹಿಟಾಚಿಯಲ್ಲಿ ಬೆಂಕಿ ಕಾಣಿಸಿಕೊಂಡ ತಕ್ಷಣವೇ ಚಾಲಕ ವಾಹನದಿಂದ ಕೆಳಗಿಳಿದಿದ್ದು, ಅಷ್ಟರಲ್ಲಾಗಲೇ ಯಂತ್ರ ಧಗಧಗನೆ ಬೆಂಕಿಗಾಹುತಿಯಾಗಿದೆ',
  '4168',
  '',
  '',
  '4168'],
 ['ಯಂತ್ರ',
  'ಲಭ್ಯವಿರುವ ಆಮದು ಯಂತ

Accuracy of the WSD System

In [28]:
def accuracy_wsd(rows):
  count = 0
  for i in range(1,len(rows)):
    if rows[i][5] == rows[i][2] or rows[i][5] == rows[i][3] or rows[i][5] == rows[i][4]:
      count = count+1
  return count/(len(rows)-1)

In [29]:
accuracy_wsd(rows)

0.54